In [2]:
"""
Running OC-SVM for anomaly detection using various frameworks

https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_anomaly_comparison.html#sphx-glr-auto-examples-miscellaneous-plot-anomaly-comparison-py



"""
import numpy as np
from matplotlib import pyplot as plt
from sklearn import svm
import pandas as pd

In [53]:
# Import X and Y data
x_data = pd.read_csv("X_data.csv")
y_data = np.loadtxt(open("Y_data.csv","rb"), delimiter=",")
y_labels = pd.read_csv("Y_labels.csv")
# Show first 5 rows of the data and print size

print(y_data.shape)

x_data.head()
y_labels.head()

(47,)


,carbonZ_2018-07-18-12-10-11_no_ground_truth
0,carbonZ_2018-07-18-15-53-31_1_engine_failure
1,carbonZ_2018-07-18-15-53-31_2_engine_failure
2,carbonZ_2018-07-18-16-22-01_engine_failure_wit...
3,carbonZ_2018-07-18-16-37-39_1_no_failure
4,carbonZ_2018-07-18-16-37-39_2_engine_failure_w...


In [71]:
# First we will run OC-SVM in a unsupervised environment (inliers and outliers together no labels)
# Fit model
outliers_fraction = 0.80
svm_obj = svm.OneClassSVM(
    kernel='rbf',
    nu = outliers_fraction,
    gamma = 0.1)

# 1: inliers, -1: Outliers
# 10 Samples are Non-failure
# 37 are Failure
x_np = x_data.to_numpy()
print(x_np.ndim)
y_pred = svm_obj.fit(x_np).predict(x_np)

2


In [65]:
n_error_outliers = y_pred[y_pred == 1].size
print(n_error_outliers)

15


In [15]:
# Helper functions to get error types and performance statistics
def get_accuracy(y_truth, y_est, sample_size):
    """
        Function to compute accuracy on how well the classification algorithm works

        Returns accuracy
    """
    acc = 0
    for i in range(len(y_truth)):
        acc = acc + (y_truth[i] == y_est[i])
    acc = (1/sample_size) * acc

    return acc

In [66]:
acc = get_accuracy(y_data, y_pred, y_data.size)

IndexError: index 46 is out of bounds for axis 0 with size 46

In [69]:
y_pred.size

46

In [68]:
x_np.size

1150